In [ ]:
import pandas as pd
fake_las_kgl = pd.read_csv('fake_data/fake_data_collect_kgl_1.csv', delimiter=';')
fake_las_kgl.head()

In [ ]:
print(fake_las_kgl.shape)
import pandas as pd
import numpy as np
np.random.seed(10)

remove_n = 2500
drop_indices = np.random.choice(fake_las_kgl.index, remove_n, replace=False)
fake_las_kgl = fake_las_kgl.drop(drop_indices)

print(fake_las_kgl.shape)

In [ ]:
fake_las_aps = pd.read_csv('fake_data/fake_data_collect_aps_1.csv', delimiter=';')
fake_las_aps.head()

In [ ]:
print(fake_las_aps.shape)
import pandas as pd
import numpy as np
np.random.seed(10)

remove_n = 2500
drop_indices = np.random.choice(fake_las_aps.index, remove_n, replace=False)
fake_las_aps = fake_las_aps.drop(drop_indices)

print(fake_las_aps.shape)

In [ ]:
fake_aps_kgl = pd.merge(fake_las_aps, fake_las_kgl, on=['wc'])

In [ ]:
fake_aps_kgl.shape

In [ ]:
y_values = fake_aps_kgl['wc']
y_values.shape

In [ ]:
del fake_aps_kgl['wc']
fake_aps_kgl.shape

In [ ]:
import numpy as np
x_values = np.array(fake_aps_kgl)
print(x_values.shape)

In [ ]:
fake_aps_kgl.head()

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
x_train, x_hold, y_train, y_hold = train_test_split(x_values, y_values, test_size=0.5, random_state=17)

In [ ]:
print(min(y_values))
print(max(y_values))
print(max(y_values) - min(y_values))

In [ ]:
x_train.shape[1]

In [ ]:
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Устанавливаем seed для повторяемости результатов
np.random.seed(42)
# Создаем последовательную модель
model = Sequential()

model.add(Dense(250, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dense(200, activation='relu', input_shape=(250,)))
model.add(Dense(100, activation='relu', input_shape=(200,)))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu', input_shape=(100,)))
model.add(Dense(49, activation='relu', input_shape=(100,)))
model.add(Dense(1))

# Компилируем модель
model.compile(loss="mse", optimizer="adam", metrics=['mae'])

print(model.summary())

In [ ]:
# Обучаем сеть
model.fit(x_train, y_train, batch_size=5, nb_epoch=50, verbose=1)

In [ ]:
y_predict = model.predict(x_hold)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

def data_analize_func(predict_for_analize, file_name = ''):
    # подготовка данных
    data_for_analize = predict_for_analize.sort_values(by=['y_hold'])
    data_for_analize['x_axis'] = [x for x in range(len(predict_for_analize['y_hold']))]
    data_for_analize.head()
    if file_name != '':
        data_for_analize.to_csv('fake_data/' + file_name, index=False, sep = ';')
    
    # построение графиков
    plt.scatter(data_for_analize['x_axis'], data_for_analize['y_predict'], color = 'blue')
    plt.scatter(data_for_analize['x_axis'], data_for_analize['y_hold'], color = 'red')
    plt.show()
    
    # regression metrics
    print('MSE                      = '+ str(mean_squared_error(data_for_analize['y_hold'], 
                                                                data_for_analize['y_predict'])))
    print('MAE                      = '+ str(mean_absolute_error(data_for_analize['y_hold'], 
                                                                 data_for_analize['y_predict'])))
    print('r2_score                 = '+ str(r2_score(data_for_analize['y_hold'], 
                                                      data_for_analize['y_predict'])) + '    (best_value is 1)')
    print('explained_variance_score = '+ str(explained_variance_score(data_for_analize['y_hold'], 
                                                                      data_for_analize['y_predict'])) + '    (best_value is 1)')

In [ ]:
predict_for_analize_fake = pd.DataFrame({'y_hold':y_hold, 'y_predict':y_predict[:,0]})
data_analize_func(predict_for_analize_fake)

In [ ]:
# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("fake_data/fake_aps_kgl_model.json", "w")
json_file.write(model_json)
json_file.close()

model.save_weights("fake_data/fake_aps_kgl_model.h5")

In [ ]:
real_aps_kgl = pd.read_csv('fake_data/real_kgl_aps.csv', delimiter=';')
real_aps_kgl.head()

In [ ]:
real_y_values = real_aps_kgl['wc']
real_y_values.shape

In [ ]:
# del real_aps_kgl['wc']
real_aps_kgl.shape

import numpy as np
real_x_values = np.array(real_aps_kgl)
print(real_x_values.shape)

In [ ]:
real_y_predict = model.predict(real_x_values)

In [ ]:
predict_for_analize_real = pd.DataFrame({'y_hold':real_y_values, 'y_predict':real_y_predict[:,0]})
data_analize_func(predict_for_analize_real)